# Optimization

A good reference is Jorge Nocedal and Stephen J. Wright, _Numerical Optimization_, Second ed., Springer.


We will use SciPy optimization routines, so let us first import it. 

Unlike NumPy, we do not import the whole SciPy. Here we specifically import _optimize_ from SciPy. <https://docs.scipy.org/doc/scipy/reference/api.html#guidelines-for-importing-functions-from-scipy>



In [2]:
from scipy import optimize 
import numpy as np # We will use numpy.array

The simplest optimization problem is an _unconstrained optimization_ that is formulated as

$$ \min_{x\in \mathbb{R}^N} f(x).$$

(Without loss of generality, we focus on minimization problems; maximizing $f$ is the same as minimizing $-f$.)



When the function $f$ is convex and differentiable, finding a minimizer is the same as finding the root of $f'$. But in general $f$ may have local minimizers.

Broadly speaking, there are two strategies to generate a sequence of candidate solution $\{x_n\}$. 

In the **line search** strategy, the algorithm first chooses a search direction, $p_n$, toward which the iterate $x_n$ is updated, and the step size $\alpha$ is chosen so as to "loosely" minimize

$$ \phi(\alpha) := f(x_n + \alpha p_n).$$

Minimizing this function exactly is costly, so $\alpha$ is chosen so that

1. $f$ decreases sufficiently, and
2. $\phi'(\alpha)$ is not too negative (or it is close to zero, or $\alpha$ is not too small).  

Algorithms are differnt in their way to choose the search direction. The steepest descent method uses

$$ p_n = -  \nabla f(x_n),$$

the Newton's method uses

$$ p_n = -  (\nabla^2 f(x_n))^{-1}\nabla f(x_n),$$

where $\nabla^2 f(x_n)$ is the Hessian, and quasi-Newton methods use

$$ p_n = -  B_n^{-1}\nabla f(x_n),$$

where $B_n$ is an approximation to the Hessian. The BFGS method is a quasi-Newton method.


The other strategy is the **trust-region** strategy. At each $n$, it approximates $f$ in a neighborhood of $x_n$ (or, in a "trust region") and then find the candidate step by solving

$$ \min_p m_n(x_n+p)$$

where $x_n+p$ lies inside the trust region. Usually $p^T p<\Delta$ is used for a trust region. In the trust-region method, therefore, both the direction and the step size are simultaneously determined.

The model $m_n$ is usually a quadratic function of the form:

$$ m_n(x_n+p) = f(x_n) + p^T \nabla f(x_n) + \frac{1}{2}p^T B_n p. $$

The matrix $B_n$ is either the Hessian or some approximation to it.



A usual test function is the Rosenbrock function. 

SciPy has it, along with its gradient and Hessian functions.



In [3]:
x0 = np.array([1.3, 0.7, 0.8, 1.9, 1.2])

# BFGS method
res = optimize.minimize(optimize.rosen, x0, method='BFGS',
               jac=optimize.rosen_der,
               options={'gtol': 1e-6, 'disp': True})

print('-----')
print(res)


Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 26
         Function evaluations: 31
         Gradient evaluations: 31
-----
      fun: 1.4759786947522541e-15
 hess_inv: array([[ 0.00763624,  0.0125159 ,  0.02359808,  0.0465684 ,  0.09317974],
       [ 0.0125159 ,  0.02489687,  0.04727754,  0.09353774,  0.18708048],
       [ 0.02359808,  0.04727754,  0.09483095,  0.18783847,  0.37561493],
       [ 0.0465684 ,  0.09353774,  0.18783847,  0.37715312,  0.75414172],
       [ 0.09317974,  0.18708048,  0.37561493,  0.75414172,  1.51296479]])
      jac: array([  9.93918700e-07,   4.21980188e-07,   2.23775033e-07,
        -6.10304485e-07,   1.34057054e-07])
  message: 'Optimization terminated successfully.'
     nfev: 31
      nit: 26
     njev: 31
   status: 0
  success: True
        x: array([ 1.,  1.,  1.,  1.,  1.])


SciPy.optimize.minimize() function has methods that are not derivative-based. One example is the Nelder-Mead revised simplex method.

In [ ]:
x0 = np.array([1.3, 0.7, 0.8, 1.9, 1.2])

# Nelder-Mead method
res = optimize.minimize(optimize.rosen, x0, method='Nelder-Mead',
               options={'xtol': 1e-8,'disp': True})

print('-----')
print(res)


## Constrained optimization

A general constrained minimization problem is given by:

$$ \min_{x \in \mathbb{R}^N} f(x) $$

subject to:

1. Equality constraints: for $i\in \mathcal{E}$, $ c_i(x) = 0$, and

2. Inequality constraints: for $i\in \mathcal{I}$, $c_i(x) \le 0$.

(We may separately have the bound constraints, $lb \le x \le ub$.)




### Linear programming

When both objective and all constraint functions are linear in x, the problem can be written as 

$$ \min_{x \in \mathbb{R}^N} c^T x $$

subject to:

1. Equality constraints: $A_{eq}x = b_{eq}$, and

2. Inequality constraints:  $A_{ub}x \le b_{ub}$.

The SciPy routine _scipy.optimize.linprog()_ also takes the bound constraints.

#### Linear programming example

Consider the following example taken from the documentation: <https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.linprog.html>

$x \in \mathbb{R}^2$.


Minimize: $-x[0] + 4x[1]$

Subject to: 
\begin{eqnarray}
-3x[0] + x[1] &\le& 6\\
x[0] + 2x[1] &\le & 4\\
x[1] &\ge& -3.
\end{eqnarray}

There is no bound constraint for $x[0]$.

In [4]:

# Parameter specification
c = [-1, 4]
A = [[-3, 1], [1, 2]]
b = [6, 4]
x0_bounds = (None, None)   # None is used to mean "no constraint"
x1_bounds = (-3, None)

# Call optimize.linprog
res = optimize.linprog(c, A_ub=A, b_ub=b, bounds=(x0_bounds, x1_bounds),
              options={"disp": True})

print('-------')
print(res)

Optimization terminated successfully.
         Current function value: -22.000000  
         Iterations: 1
-------
     fun: -22.0
 message: 'Optimization terminated successfully.'
     nit: 1
   slack: array([ 39.,   0.])
  status: 0
 success: True
       x: array([ 10.,  -3.])


Here, _linprog_ returns an object that contains all kinds of information. To see the solution only, we can call:

In [ ]:
res.x

## Quadratic programming

The general quadratic problem (QP) is stated as:

$$ \min_{x \in \mathbb{R}^N} x^T G x + x^Tc $$

subject to:

1. Equality constraints: $A_{eq}x = b_{eq}$, and

2. Inequality constraints:  $A_{ub}x \le b_{ub}$.

When $G$ is a positive semi-definite, the problem is called a convex QP.

Problems of this class can be solved in a finite amount of computation. For example, in the absence of inequality constraints, the first-order necessary conditions are linear and, hence, finding a stationary point of the Lagrangian amounts to solving a system of linear equations. 

### Sequential quadratic programming

Sequential quadratic programming generates a sequence $\{x_n\}$ by, for each $n$, solving a QP whose objective function quadratically approximates the Lagrangian and constraints linearly approximate the original constraint around $x_n$, to determine the search direction. The step size may be determined by line search. 

In SciPy, this can be used by calling scipy.optimize.minimize() by setting a keyword argument _method = 'SLSQP'_.



The following example is taken from <https://docs.scipy.org/doc/scipy/reference/tutorial/optimize.html>:

$$ \max_{x,y} f(x, y) = 2 x y + 2 x - x^2 - 2 y^2$$

subject to 

\begin{eqnarray}
x^3−y &=&0\\
y−1 &\ge &0
\end{eqnarray}

In [ ]:
def func(x, sign=1.0):
    """ Objective function """
    return sign*(2*x[0]*x[1] + 2*x[0] - x[0]**2 - 2*x[1]**2)

# When minimizing, sign=-1.0 will be specified.

def func_deriv(x, sign=1.0):
    """ Derivative of objective function """
    dfdx0 = sign*(-2*x[0] + 2*x[1] + 2)
    dfdx1 = sign*(2*x[0] - 4*x[1])
    return np.array([ dfdx0, dfdx1 ])

The constraint must be supplied in the form of _dictionary_.

In [ ]:
cons = ({'type': 'eq',
         'fun' : lambda x: np.array([x[0]**3 - x[1]]),
         'jac' : lambda x: np.array([3.0*(x[0]**2.0), -1.0])},
        {'type': 'ineq',
         'fun' : lambda x: np.array([x[1] - 1]),
         'jac' : lambda x: np.array([0.0, 1.0])})

In [ ]:
res = optimize.minimize(func, [-1.0,1.0], args=(-1.0,), jac=func_deriv, method='SLSQP', options={'disp': True})

print('-----')
print(res)


## Augmented Lagrangian method

Though not implemented in SciPy.optimize, the Augmented Lagrangian method is available in e.g. NLopt <http://ab-initio.mit.edu/wiki/index.php/NLopt> For installation via conda, see <https://anaconda.org/conda-forge/nlopt>.

The basic idea behind it can be intuitively understood by studying the penalty function method first.

### Penalty function approach

Quite obviously, unconstrained optimization is much simpler than constrained optimization. 

The penalty function approach basically tries to approximate a latter problem by a former one. 

Consider first the constraint minimization problem above, without inequality constraints.

The quadratic penalty function $Q(x;\mu)$ is defined as

$$ Q(x;\mu) = f(x) + \frac{\mu}{2} \sum_{i\in \mathcal{E}} \left(c_i(x)\right)^2, $$

where $mu>0$ is the penalty parameter.

What we expect is that a minimizer of $Q$ given $\mu$ converges to a solution to the original constrained problem as $\mu \uparrow \infty$. 

When inequaility constraints exist, one can define $Q$ as

$$ Q(x;\mu) = f(x) + \frac{\mu}{2} \sum_{i\in \mathcal{E}} \left(c_i(x)\right)^2 + \frac{\mu}{2}\sum_{i\in\mathcal{I}}\left(\max\{0,-c_i(x)\}\right)^2. $$

Instead of a quadratic function, one may use the absolute value $|.|$ (the $l^1$-penalty function).

There is a problem, however. As $\mu$ tends to infinity, the Hessian of $Q$ becomes ill-conditioned (some eigenvalues grow with $\mu$), which makes the calculation of the Newton step less accurate. In the Augmented Lagrangian method $\mu$ does not have to grow indefinitely, and this problem is less severe. 

### Augmented Lagrangian method

For simplicity, consider again the problem without inequality constraints. The augmented Lagrangian is defined as 

$$ \mathcal{L}_A(x;\lambda,\mu) = f(x) -\sum_{i\in\mathcal{E}}\lambda_ic_i(x)  + \frac{\mu}{2} \sum_{i\in \mathcal{E}} \left(c_i(x)\right)^2.$$

I.e. we augment the Lagrangian with the quadratic penalty function. Again, to minimize $\mathcal{L}_A$ with respect to $x$ for given $(\lambda,\mu)$ only requires unconstrained minimization routines (possibly with ones that solve problems with the bound constraints). 

Theoretically it is shown that, if $\lambda = \lambda^*$ (true multiplier), then there is a threshold $\overline{\mu}$ such that, for all $\mu \ge \overline{\mu}$, a minimizer of $\mathcal{L}_A$ is $x^*$, a solution to the original problem. 

In practice, we do not know true $\lambda^*$. The method threfore also specifies how the estimate of $\lambda$, as well as $x$ and $\mu$, is updated as we iterate. 


## Appendix: Active set methods and interior point methods

The biggest difficulty in dealing with inequality constraints is that we do not know a priori which constraints bind and which do not, and that we need to figure out which ones are active/binding. 

Active set methods make educated guess about which inequality constraints are going to bind/be active, and keep updating the guess along iteration. 

Interior point methods, or barrier methods, take a different approach. They alter the problem so that inequality constraints are always satisfied with strict inequalities (i.e. we are always in the, sort of, interior of the constraint set). 

$$ \min_{x \in \mathbb{R}^N, \{s_i\}_{i \in \mathcal{I} } } f(x) - \mu\sum_{i\in \mathcal{I}} \log s_i $$

subject to


1. Equality constraints: for $i\in \mathcal{E}$, $ c_i(x) = 0$, and

2. Inequality constraints: for $i\in \mathcal{I}$, $c_i(x) - s_i \le 0$.

for some $\mu>0$. 

$\{s_i\}_{i\in \mathcal{I}}$ are called slack variables. The structure of the above problem implies that its solution satisfies $s_i>0$ for all $i$.

The basic idea is to consider a sequence $\{\mu_k\}$ that converges to zero, and to generate a solution to the above problem $\{x_k\}$.


# Quiz
## 1. 

Suppose we want to solve

$$ \min_{x \in \mathbb{R}^N} f(x)$$

subject only to the bound constraints:

$$ lb_i \le x_i \le ub_i $$

for $i=1,2,...,N$. 

Which methods can we use in _scipy.optimize.minimize()_ to solve this problem? 

Hint: In the documentation <https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.minimize.html>, click on (See here) next to the method names to check whether each method takes "bounds" as input. 



## 2.

In _scipy.optimize.minimize()_, we need to specify the set of constraints as a dictionary or a sequence of dictionaries. 

Read the "The Dictionary Data Type" and "Dictionaries vs. Lists" sections in <https://automatetheboringstuff.com/chapter5/> and answer Practice Questions Q1-Q4. 


## 3. 

_scipy.optimize.minimize()_ function returns an _OptimizeResult_ object that contains a lot of useful information. 

In the lecture notebook, we have assigned the result to the variable _res_ as follows:


How do we know whether an algorithm has ended successfully or not? 

Hint: In the lecture we have displayed a solution by executing: _print(res.x)_ 

Hint: Read the "Attributes" section in <https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.OptimizeResult.html> to see whether there are any attributes that tell you whether an algorithm has finished successfully.



## 4.
The Simplex Algorithm is the most popular method to sove the linear programming problem <https://en.wikipedia.org/wiki/Simplex_algorithm>. Explain the algorithm briefly.